Correlation plot is between one population and two varaibles of comparison
T - tests compare means between two populations and one variable of comparison


stats.ttest_ind(df[df.State == 'UT'].Score, df[df.State =='UT'].Score)

        http://localhost:8888/tree?token=bbd94d3c8195a226912c267fd29c4a6191bbfd8ecd3afb6a

        crime-housing-austin-2015.csv
0.05 p - vaue is saying that there is a 1 in 20 chance that

A p-value less than 0.05 is typically considered to be statistically significant, in which case the null hypothesis should be rejected. A p-value greater than 0.05 means that deviation from the null hypothesis is not statistically significant, and the null hypothesis is not rejected




Potential t - test: two separate locations (or types of locations) and comparing a ratio of median home income and unemployment

Potential p - value test: median income and unemployment for all zip codes with murder 

In [ ]:
import pandas as pd

import seaborn as sns

import numpy as np
from matplotlib import pyplot as plt
from scipy import stats

dataset = pd.read_csv('crime-housing-austin-2015.csv')
dataset['Medianhouseholdincome'] = dataset['Medianhouseholdincome'].str.replace('$', '').astype('float')
dataset['Unemployment'] = dataset['Unemployment'].str.replace('%', '').astype('float')
dataset['Medianrent'] = dataset['Medianrent'].str.replace('$', '').astype('float')

dataset['Percentageofrentalunitsinpoorcondition'] = dataset['Percentageofrentalunitsinpoorcondition'].str.replace('%', '').astype('float')
dataset= dataset.dropna()



datasetGrouped = dataset.groupby(['Zip_Code_Crime']).agg({'Medianhouseholdincome':'first', 'Unemployment':'first','Populationwithdisability':'first','Medianrent':'first', 'Percentageofrentalunitsinpoorcondition' : 'first'}).reset_index()
lowIncome = dataset[dataset.Medianhouseholdincome <= 30000]





lowIncomeAndHighUnemployment = lowIncome[lowIncome.Unemployment < 12]
##len(lowIncomeAndHighUnemployment)




In [ ]:
occurrences_per_value= dataset['Zip_Code_Crime'].value_counts().reset_index()

mergedDatasets = pd.merge(occurrences_per_value, datasetGrouped, on='Zip_Code_Crime')


sns.regplot(x='count', y='Unemployment',data=mergedDatasets)


In [ ]:
## open up the other dataset

zipCodes = pd.read_csv('AustinZipCodes.csv')
zipCodes['Population'] = zipCodes['Population'].str.replace(',', '').astype('float')

zipCodes['Population'] = zipCodes['Population'].astype(int)
mergedDatasets.rename(columns={'Zip_Code_Crime': 'Zip Code'}, inplace=True)

zipCodesPopulationAndCrimes = pd.merge(mergedDatasets, zipCodes, on='Zip Code')



In [ ]:

zipCodesPopulationAndCrimes['Crime/Population'] = zipCodesPopulationAndCrimes['count'] / zipCodesPopulationAndCrimes['Population']




zipCodesPopulationAndCrimes.rename(columns={'Percentageofrentalunitsinpoorcondition': '% Rental Units in Poor Condition'}, inplace=True)
#sns.regplot(x='Crime/Population', y='% Rental Units in Poor Condition',data=zipCodesPopulationAndCrimes)
display(stats.pearsonr(zipCodesPopulationAndCrimes['% Rental Units in Poor Condition'], zipCodesPopulationAndCrimes['Crime/Population']))

zipCodesPopulationAndCrimes = zipCodesPopulationAndCrimes[zipCodesPopulationAndCrimes['Crime/Population'] < 0.5]

sns.regplot(x='Crime/Population', y='% Rental Units in Poor Condition',data=zipCodesPopulationAndCrimes)
plt.title('Percentage of Units in Poor Condition to Crime Per Population In Each Zip Code')
display(stats.pearsonr(zipCodesPopulationAndCrimes['% Rental Units in Poor Condition'], zipCodesPopulationAndCrimes['Crime/Population']))
